In [ ]:
!pip install ipython-sql
!pip install sqlalchemy
!pip install plotly

In [47]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import warnings
from pivottablejs import pivot_ui
import urllib
from sqlalchemy import create_engine
import plotly.graph_objects as go
import plotly.express as px

In [109]:
query_accidentes = """
    select 
		 cast(a.color_veh as nvarchar)+ ' - '+ f.descripcion as color_vehiculo
		,sum(e.codigo) as cantidad
	from
		fallecidos_lesionados as a
	inner join vehiculos_involucrados as b on a.num_corre = b.num_corre
	inner join hechos_transito as c on a.num_corre = c.num_corre
	inner join tipo_vehiculo as d on a.tipo_veh = d.codigo
	inner join resultado_accidente as e on a.fall_les = e.codigo
	inner join color_vehiculo as f on a.color_veh = f.codigo
	where
        a.tipo_veh not in (99)
	and a.color_veh in (1,5)
	group by
		a.color_veh,f.descripcion
	order by
		a.color_veh asc
"""

In [112]:
query_fallecidos = """
	select 
		 cast(a.color_veh as nvarchar)+ ' - '+ f.descripcion as color_vehiculo
		,sum(e.codigo) as cantidad
	from
		fallecidos_lesionados as a
	inner join vehiculos_involucrados as b on a.num_corre = b.num_corre
	inner join hechos_transito as c on a.num_corre = c.num_corre
	inner join tipo_vehiculo as d on a.tipo_veh = d.codigo
	inner join resultado_accidente as e on a.fall_les = e.codigo
	inner join color_vehiculo as f on a.color_veh = f.codigo
	where
        a.tipo_veh not in (99)
	and e.codigo = 1
	and a.color_veh in (1,5)
	group by
		a.color_veh,f.descripcion
	order by
		a.color_veh asc
"""

In [113]:
query_lesionados = """
	select 
		 cast(a.color_veh as nvarchar)+ ' - '+ f.descripcion as color_vehiculo
		,sum(e.codigo) as cantidad
	from
		fallecidos_lesionados as a
	inner join vehiculos_involucrados as b on a.num_corre = b.num_corre
	inner join hechos_transito as c on a.num_corre = c.num_corre
	inner join tipo_vehiculo as d on a.tipo_veh = d.codigo
	inner join resultado_accidente as e on a.fall_les = e.codigo
	inner join color_vehiculo as f on a.color_veh = f.codigo
	where
        a.tipo_veh not in (99)
	and e.codigo = 2
	and a.color_veh in (1,5)
	group by
		a.color_veh,f.descripcion
	order by
		a.color_veh asc
"""

In [48]:
# datos = pd.read_excel('datos.xlsx', sheet_name='transito')

In [66]:
params ='Trusted_Connection=Yes;'\
        'DRIVER={ODBC Driver 13 for SQL Server};' \
         'SERVER=PC-PSAO\MSSQL2017;' \
         'PORT=1433;' \
         'DATABASE=db_transito;' \
         'UID=sa;' \
         'PWD=123456;'
            
params = urllib.parse.quote_plus(params)

In [67]:
db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

# Explorando Lesionados y Fallecidos por Vehiculo

In [68]:
sql = 'EXEC [dbo].[stp_S_total_accidentes_tipo_vehiculo] '

accidentes_vehiculos = pd.read_sql_query(sql, db)

In [69]:
fig = px.pie(accidentes_vehiculos, values='cantidad', names='vehiculo',
             title='Lesionados y Fallecidos por Vehiculo',
             hover_data=['cantidad'], labels={'cantidad':'Fallecidos y Lesionados'})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [70]:
 accidentes_vehiculos

,vehiculo,cantidad
0,1 - Automóvil,1553
1,2 - Camioneta sport o blazer,602
2,3 - Pick up,1651
3,4 - Motocicleta,4906
4,5 - Camión,474
5,6 - Cabezal,216
6,7 - Bus extraurbano,274
7,8 - Jeep,20
8,9 - Microbús,502
9,10 - Taxi,27


## Analizando Fallecidos por Vehiculo

In [71]:
sql = 'EXEC [dbo].[stp_SF_total_accidentes_tipo_vehiculo] 1'

fallecidos_vehiculo = pd.read_sql_query(sql, db)

In [72]:
fallecidos_vehiculo

,vehiculo,cantidad
0,1 - Automóvil,105
1,2 - Camioneta sport o blazer,32
2,3 - Pick up,125
3,4 - Motocicleta,398
4,5 - Camión,64
5,6 - Cabezal,34
6,7 - Bus extraurbano,8
7,8 - Jeep,2
8,9 - Microbús,28
9,10 - Taxi,5


## Analizando Lesionados por Vehículo 

In [73]:
sql = 'EXEC [dbo].[stp_SF_total_accidentes_tipo_vehiculo] 2'

lesionado_vehiculo = pd.read_sql_query(sql, db)

In [74]:
lesionado_vehiculo

,vehiculo,cantidad
0,1 - Automóvil,1448
1,2 - Camioneta sport o blazer,570
2,3 - Pick up,1526
3,4 - Motocicleta,4508
4,5 - Camión,410
5,6 - Cabezal,182
6,7 - Bus extraurbano,266
7,8 - Jeep,18
8,9 - Microbús,474
9,10 - Taxi,22


In [75]:
fig = go.Figure(data=[
    go.Bar(name='Fallecidos',
           x=fallecidos_vehiculo['vehiculo'],
           y=fallecidos_vehiculo['cantidad'],
           text=dfallecidos_vehiculo['cantidad'],
           marker_color='rgb(0,100,0)'),
    
    go.Bar(name='Lesionados',
           x=lesionado_vehiculo['vehiculo'],
           y=lesionado_vehiculo['cantidad'],
           text=lesionado_vehiculo['cantidad'],
           marker_color='rgb(255,127,80)')
])
# Change the bar mode
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(barmode='group',title='Fallecidos y Lesionados por Tipo de Vehiculo en el 2018',xaxis_title="Color Vehículo",
    yaxis_title="Tipo Vehículo")
fig.show()

Debido a que nos centraremos en identificar la relación de accidentes de los vehículos de color rojo, vamos a omitir los datos que estan catalogados como ignorado (código 99)

# Fallecidos y Lesionados por Color de Vehículos

### Fallecidos por Color Vehículo

In [76]:
sql = 'EXEC [dbo].[stp_S_total_accidentes_color_vehiculo] 1'

dfallecidos_vehiculo = pd.read_sql_query(sql, db)

In [77]:
dfallecidos_vehiculo

,color_vehiculo,cantidad
0,1 - Rojo,125
1,2 - Blanco,114
2,3 - Azul,55
3,4 - Gris,81
4,5 - Negro,186
5,6 - Verde,31
6,7 - Amarillo,17
7,8 - Celeste,10
8,9 - Corinto,25
9,10 - Café,1


### Lesionados por Color de Vehículo

In [78]:
sql = 'EXEC [dbo].[stp_S_total_accidentes_color_vehiculo] 2'

dlesionados_vehiculo = pd.read_sql_query(sql, db)

In [79]:
dlesionados_vehiculo

,color_vehiculo,cantidad
0,1 - Rojo,1898
1,2 - Blanco,1468
2,3 - Azul,916
3,4 - Gris,978
4,5 - Negro,1982
5,6 - Verde,434
6,7 - Amarillo,240
7,8 - Celeste,116
8,9 - Corinto,292
9,10 - Café,44


In [80]:
fig = go.Figure(data=[
    go.Bar(name='Fallecidos',
           x=dfallecidos_vehiculo['color_vehiculo'],
           y=dfallecidos_vehiculo['cantidad'],
           text=dfallecidos_vehiculo['cantidad'],
           marker_color='rgb(0,100,0)'),
    
    go.Bar(name='Lesionados',
           x=dlesionados_vehiculo['color_vehiculo'],
           y=dlesionados_vehiculo['cantidad'],
           text=dlesionados_vehiculo['cantidad'],
           marker_color='rgb(255,127,80)')
])
# Change the bar mode
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(barmode='group',title='Fallecidos y Lesionados por Color de Vehiculo en el 2018',xaxis_title="Color Vehículo",
    yaxis_title="Personas")
fig.show()


### Revisando unicamente datos de fallecidos 

In [81]:
fig = px.bar(dfallecidos_vehiculo, y='cantidad', x='color_vehiculo', text='cantidad')

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update_layout(title='Fallecidos por Color de Vehiculo en el 2018',xaxis_title="Color Vehículo",
    yaxis_title="Personas Fallecidas",uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()

### Revisando unicamente datos de lesionados 

In [82]:
fig = px.bar(dlesionados_vehiculo, y='cantidad', x='color_vehiculo', text='cantidad')

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update_layout(title='Lesionados por Color de Vehiculo en el 2018',xaxis_title="Color Vehículo",
    yaxis_title="Personas Lesionadas",uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()

# Fallecidos por Color y Tipo de Vehículo

In [83]:
sql = 'EXEC [dbo].[stp_S_accidentes_por_vehiculo] 1'

datos = pd.read_sql_query(sql, db)

In [84]:
datos

,Vehiculo,Verde,Varios colores,Turquesa,Rosado,Rojo,Negro,Morado,Marfil,Ignorado,Gris,Corinto,Celeste,Café,Blanco,Beige,Azul,Anaranjado,Amarillo
0,1 - Automóvil,3,0,0,0,5,14,1,0,35,27,6,1,0,5,3,3,1,1
1,2 - Camioneta sport o blazer,2,0,0,0,2,7,0,0,3,7,2,1,1,4,1,2,0,0
2,3 - Pick up,12,0,0,0,19,15,0,0,21,22,7,0,0,19,2,8,0,0
3,4 - Motocicleta,3,6,0,0,78,140,0,0,80,17,6,0,0,28,0,33,4,3
4,5 - Camión,1,0,0,0,1,1,0,0,12,0,0,1,0,41,0,4,1,2
5,6 - Cabezal,1,0,0,0,6,3,0,0,5,0,0,0,0,7,0,2,9,1
6,7 - Bus extraurbano,1,0,0,0,1,1,0,0,3,0,0,0,0,0,0,0,0,2
7,8 - Jeep,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
8,9 - Microbús,2,0,0,0,0,0,0,0,10,7,0,2,0,4,1,1,0,1
9,10 - Taxi,0,0,0,0,0,1,0,0,0,0,4,0,0,0,0,0,0,0


In [85]:

#colores = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
#fig = go.Figure()

#c_color = 0
#for col in datos.columns: 
#    if(col != 'Vehiculo'):
#        columnas.append(col)
        
#        fig.add_trace(go.Bar(
#            x=datos['Vehiculo'],
#            y=datos[col],
#            name=col,
#            marker_color=colores[c_color]
#        ))
#    c_color += 1
        
#fig.update_layout(barmode='group', xaxis_tickangle=-45)
#fig.show()

In [86]:
fig = px.pie(datos, values='Rojo', names='Vehiculo',
             title='Fallecidos por Tipo de Vehiculo Rojos',
             hover_data=['Rojo'], labels={'Rojo':'Pallecidos'})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

#### Hay datos que no aparecen en el grafico de pie, se realizara una grafica de barras para poder ver toda la informacion 

In [87]:
fig = px.bar(datos, y='Rojo', x='Vehiculo', text='Rojo')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title='Fallecidos por Tipo de Vehiculo Rojo',uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [88]:
fig = px.bar(datos, y='Negro', x='Vehiculo', text='Negro')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title='Fallecidos por Tipo de Vehiculo Negro',uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [89]:
fig = px.bar(datos, y='Rojo', x='Vehiculo', text='Rojo')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title='Fallecidos por Tipo de Vehiculo Rojos',uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

# Lesionados por Color y Tipo de Vehículo

In [90]:
sql = 'EXEC [dbo].[stp_S_accidentes_por_vehiculo] 2'

datosl = pd.read_sql_query(sql, db)

In [91]:
datosl

,Vehiculo,Verde,Varios colores,Turquesa,Rosado,Rojo,Negro,Morado,Marfil,Ignorado,Gris,Corinto,Celeste,Café,Blanco,Beige,Azul,Anaranjado,Amarillo
0,1 - Automóvil,106,0,0,4,86,148,2,0,228,348,74,32,4,266,54,68,0,28
1,2 - Camioneta sport o blazer,60,0,0,0,36,82,0,0,44,118,38,20,6,58,40,48,0,20
2,3 - Pick up,74,8,0,0,242,224,0,0,144,186,106,24,22,226,32,208,0,30
3,4 - Motocicleta,62,42,0,0,1126,1492,0,0,754,202,52,6,6,198,0,462,58,48
4,5 - Camión,8,0,0,0,0,0,0,0,74,4,0,2,0,298,0,20,0,4
5,6 - Cabezal,4,0,0,0,4,6,0,0,20,2,8,4,0,68,2,18,44,2
6,7 - Bus extraurbano,20,2,0,0,18,10,0,0,72,0,0,0,0,6,0,40,32,66
7,8 - Jeep,0,0,0,0,0,0,0,0,0,0,0,4,0,4,2,8,0,0
8,9 - Microbús,48,0,0,0,40,0,0,0,80,106,12,16,4,116,12,38,0,2
9,10 - Taxi,0,0,0,0,0,0,0,0,4,0,0,0,0,18,0,0,0,0


In [92]:
fig = px.bar(datosl, y='Rojo', x='Vehiculo', text='Rojo')


fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title='Lesionados por Tipo de Vehiculo Rojos',uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [93]:
fig = px.bar(datosl, y='Negro', x='Vehiculo', text='Negro')


fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title='Lesionados por Tipo de Vehiculo Negro',uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

# Buscando la cantidad de fallecidos y lesionados en el grupo de hora, por color de vehiculo

## Analizando fallecidos

In [94]:
sql = 'EXEC [dbo].[stp_S_accidentes_por_grupo_hora]  1'

grupof = pd.read_sql_query(sql, db)

In [95]:
grupof

,Grupo,Verde,Varios colores,Turquesa,Rosado,Rojo,Negro,Morado,Marfil,Ignorado,Gris,Corinto,Celeste,Café,Blanco,Beige,Azul,Anaranjado,Amarillo
0,1 - 00:00 a 05:59,9,2,0,0,25,49,0,0,38,17,5,1,0,25,2,7,2,3
1,2 - 06:00 a 11:59,8,2,0,0,24,33,1,0,30,10,10,0,0,30,4,9,2,6
2,3 - 12:00 a 17:59,10,1,0,0,28,36,0,0,45,26,3,8,0,34,1,18,8,6
3,4 - 18:00 a 23:59,4,4,0,0,48,68,0,0,69,28,7,1,1,25,0,21,3,2


## Analizando lesionados  

In [96]:
sql = 'EXEC [dbo].[stp_S_accidentes_por_grupo_hora]  2'

grupol = pd.read_sql_query(sql, db)

In [97]:
grupol

,Grupo,Verde,Varios colores,Turquesa,Rosado,Rojo,Negro,Morado,Marfil,Ignorado,Gris,Corinto,Celeste,Café,Blanco,Beige,Azul,Anaranjado,Amarillo
0,1 - 00:00 a 05:59,36,24,0,0,252,284,0,0,192,208,68,20,4,202,30,154,0,72
1,2 - 06:00 a 11:59,144,72,0,0,446,412,0,0,500,210,32,22,16,308,44,188,64,22
2,3 - 12:00 a 17:59,136,12,0,2,588,602,2,0,398,314,96,48,14,594,32,284,60,92
3,4 - 18:00 a 23:59,118,28,2,4,612,684,2,0,564,246,96,26,10,364,48,290,22,54


# RESUMEN

In [114]:
res_total_accidentes = pd.read_sql_query(query_accidentes, db)

In [115]:
res_total_fallecidos = pd.read_sql_query(query_fallecidos, db)

In [116]:
res_total_lesionados = pd.read_sql_query(query_lesionados, db)

In [117]:
fig = go.Figure(data=[
    go.Bar(name='Fallecidos',
           x=res_total_fallecidos['color_vehiculo'],
           y=res_total_fallecidos['cantidad'],
           text=res_total_fallecidos['cantidad'],
           marker_color='rgb(29,152,202)'),
    
    go.Bar(name='Lesionados',
           x=res_total_lesionados['color_vehiculo'],
           y=res_total_lesionados['cantidad'],
           text=res_total_lesionados['cantidad'],
           marker_color='rgb(115,29,202)'),
    
    go.Bar(name='Fallecidos y Lesionados',
           x=res_total_accidentes['color_vehiculo'],
           y=res_total_accidentes['cantidad'],
           text=res_total_accidentes['cantidad'],
           marker_color='rgb(0,100,0)')
    

])
# Change the bar mode
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(barmode='group',title='Fallecidos en Vehiculos Rojos y Negros en el 2018',xaxis_title="Tipo Vehículo",
    yaxis_title="Personas")
fig.show()

In [118]:
fig = go.Figure(data=[
    go.Bar(name='Fallecidos - Rojos',
           x=datos['Vehiculo'],
           y=datos['Rojo'],
           text=datos['Rojo'],
           marker_color='rgb(0,100,0)'),
    
    go.Bar(name='Fallecidos - Negro',
           x=datos['Vehiculo'],
           y=datos['Negro'],
           text=datos['Negro'],
           marker_color='rgb(255,127,80)')
])
# Change the bar mode
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(barmode='group',title='Fallecidos en Vehiculos Rojos y Negros en el 2018',xaxis_title="Tipo Vehículo",
    yaxis_title="Personas")
fig.show()


In [119]:
fig = go.Figure(data=[
    go.Bar(name='Fallecidos - Rojos',
           x=datosl['Vehiculo'],
           y=datosl['Rojo'],
           text=datosl['Rojo'],
           marker_color='rgb(0,100,0)'),
    
    go.Bar(name='Fallecidos - Negro',
           x=datosl['Vehiculo'],
           y=datosl['Negro'],
           text=datosl['Negro'],
           marker_color='rgb(255,127,80)')
])
# Change the bar mode
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(barmode='group',title='Lesionados en Vehiculos Rojos y Negros en el 2018',xaxis_title="Tipo Vehículo",
    yaxis_title="Personas")
fig.show()